In [1]:
# 安装 AKShare
# !pip install akshare

import akshare as ak
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import requests

print("✅ AKShare 已导入")

✅ AKShare 已导入


In [4]:
import akshare as ak
import pandas as pd  # 用于处理数据，可选

# 获取豆粕主力连续合约 (M0) 的历史日数据
df = ak.futures_main_sina(symbol="M0", start_date="20200101")  # end_date 可选，默认为当前
print(df.head())  # 显示前几行数据
# 示例输出列：日期、开盘价、最高价、最低价、收盘价、成交量、持仓量、动态结算价

           日期   开盘价   最高价   最低价   收盘价      成交量      持仓量  动态结算价
0  2020-01-02  2780  2786  2766  2771   631920  1675405   2774
1  2020-01-03  2772  2785  2760  2761   851662  1647024   2774
2  2020-01-06  2757  2762  2728  2739  1002411  1656826   2744
3  2020-01-07  2738  2748  2736  2743   565047  1649835   2743
4  2020-01-08  2739  2742  2714  2727  1141167  1735101   2729


In [5]:
import akshare as ak

# 获取 DCE 交易所 2024 年数据，包括豆粕 (variety="M")
df = ak.get_futures_daily(start_date="20240101", end_date="20240903", market="DCE")
# 过滤豆粕数据
soybean_meal_df = df[df['variety'] == 'M']
print(soybean_meal_df.head())
# 输出列：合约代码、日期、开盘价、最高价、最低价、收盘价、成交量、持仓量、成交额、结算价、前结算价、品种

    symbol      date    open    high     low   close    volume  open_interest  \
144  m2401  20240102  3810.0  3859.0  3760.0  3770.0    1026.0         5464.0   
145  m2403  20240102  3748.0  3748.0  3667.0  3667.0  109707.0       220378.0   
146  m2405  20240102  3290.0  3294.0  3234.0  3248.0  884958.0      1429605.0   
147  m2407  20240102  3288.0  3288.0  3222.0  3235.0   41046.0       478590.0   
148  m2408  20240102  3352.0  3358.0  3298.0  3313.0   13357.0        75554.0   

       turnover  settle  pre_settle variety  
144     3883.47  3785.0      3910.0       M  
145   404981.01  3691.0      3812.0       M  
146  2883493.51  3258.0      3343.0       M  
147   133574.69  3254.0      3324.0       M  
148    44407.88  3324.0      3392.0       M  


In [6]:
def get_soybean_meal_contracts_akshare():
    """使用 AKShare 获取豆粕期货的真实多合约数据"""
    
    print("🔄 正在获取豆粕期货合约数据...")
    
    try:
        # 获取豆粕期货的所有合约基本信息
        # tool_trade_date_hist_sina() 获取交易日历
        # futures_contracts = ak.futures_main_sina(symbol="M0")  # M0代表豆粕
        # print(f"✅ 获取到豆粕期货基本信息")
        
        # 获取豆粕主力连续合约数据
        df_main = ak.futures_main_sina(symbol="M0")  # M0 是主力连续
        print(f"✅ 获取主力连续合约数据: {len(df_main)} 条记录")
        
        # 获取具体月份合约数据 (近几个月的合约)
        contract_months = ['2301', '2305', '2309', '2401', '2405', '2409', '2501']
        all_contracts_data = {}
        
        for month in contract_months:
            contract_symbol = f"M{month}"
            try:
                contract_data = ak.futures_zh_daily_sina(symbol=contract_symbol)
                if not contract_data.empty:
                    all_contracts_data[contract_symbol] = contract_data
                    print(f"   ✅ 获取合约 {contract_symbol}: {len(contract_data)} 条记录")
                else:
                    print(f"   ⚠️ 合约 {contract_symbol}: 无数据")
            except Exception as e:
                print(f"   ❌ 合约 {contract_symbol} 获取失败: {e}")
        
        return df_main, all_contracts_data
        
    except Exception as e:
        print(f"❌ 获取豆粕期货数据失败: {e}")
        return None, None

# 执行数据获取
main_contract_data, individual_contracts = get_soybean_meal_contracts_akshare()

🔄 正在获取豆粕期货合约数据...
✅ 获取主力连续合约数据: 5033 条记录
   ✅ 获取合约 M2301: 242 条记录
   ✅ 获取合约 M2305: 243 条记录
   ✅ 获取合约 M2309: 243 条记录
   ✅ 获取合约 M2401: 238 条记录
   ✅ 获取合约 M2405: 242 条记录
   ✅ 获取合约 M2409: 242 条记录
   ✅ 获取合约 M2501: 241 条记录


In [7]:
def create_real_calendar_spread_from_akshare(individual_contracts):
    """从真实多合约数据构建日历价差"""
    
    if not individual_contracts or len(individual_contracts) < 2:
        print("❌ 需要至少2个合约数据来构建价差")
        return None
    
    # 选择两个合约作为近月和远月
    contract_names = list(individual_contracts.keys())
    contract_names.sort()  # 按合约月份排序
    
    print(f"🔧 可用合约: {contract_names}")
    
    # 选择相邻的两个合约构建价差
    near_contract = contract_names[0]  # 近月
    far_contract = contract_names[1]   # 远月
    
    print(f"📊 构建价差: {near_contract} (近月) vs {far_contract} (远月)")
    
    # 获取两个合约的数据
    near_data = individual_contracts[near_contract].copy()
    far_data = individual_contracts[far_contract].copy()
    
    # 统一日期索引
    near_data['date'] = pd.to_datetime(near_data['date'])
    far_data['date'] = pd.to_datetime(far_data['date'])
    
    near_data.set_index('date', inplace=True)
    far_data.set_index('date', inplace=True)
    
    # 找到共同的交易日期
    common_dates = near_data.index.intersection(far_data.index)
    
    if len(common_dates) == 0:
        print("❌ 两个合约没有共同的交易日期")
        return None
    
    print(f"📅 共同交易日期: {len(common_dates)} 天")
    
    # 构建价差数据
    spread_data = pd.DataFrame(index=common_dates)
    spread_data['NEAR'] = near_data.loc[common_dates, 'close']  # 近月收盘价
    spread_data['FAR'] = far_data.loc[common_dates, 'close']    # 远月收盘价
    spread_data['SPREAD'] = spread_data['FAR'] - spread_data['NEAR']
    
    # 添加额外信息
    spread_data['NEAR_VOLUME'] = near_data.loc[common_dates, 'volume']
    spread_data['FAR_VOLUME'] = far_data.loc[common_dates, 'volume']
    
    print(f"✅ 价差数据构建完成: {len(spread_data)} 条记录")
    print(f"📈 价差统计: 均值={spread_data['SPREAD'].mean():.2f}, "
          f"标准差={spread_data['SPREAD'].std():.2f}")
    
    return spread_data

# 如果获取到了合约数据，构建价差
if individual_contracts:
    real_spread_data = create_real_calendar_spread_from_akshare(individual_contracts)

🔧 可用合约: ['M2301', 'M2305', 'M2309', 'M2401', 'M2405', 'M2409', 'M2501']
📊 构建价差: M2301 (近月) vs M2305 (远月)
📅 共同交易日期: 165 天
✅ 价差数据构建完成: 165 条记录
📈 价差统计: 均值=-386.16, 标准差=163.02


In [8]:
def detect_main_contract_rollover(individual_contracts):
    """检测主力合约切换点"""
    
    if not individual_contracts:
        return None
    
    print("🔍 分析主力合约切换...")
    
    # 计算每个合约的成交量和持仓量
    contract_activity = {}
    
    for contract_name, contract_data in individual_contracts.items():
        if not contract_data.empty:
            # 计算平均成交量作为活跃度指标
            avg_volume = contract_data['volume'].mean()
            contract_activity[contract_name] = avg_volume
    
    print("📊 合约活跃度 (平均成交量):")
    for contract, volume in sorted(contract_activity.items()):
        print(f"   {contract}: {volume:,.0f}")
    
    # 识别主力合约 (成交量最大的)
    main_contract = max(contract_activity, key=contract_activity.get)
    print(f"🏆 当前主力合约: {main_contract}")
    
    # 创建展期日历
    rollover_calendar = pd.DataFrame({
        'Date': [datetime.now().date()],  # 简化示例
        'OldContract': [main_contract],
        'NewContract': [list(contract_activity.keys())[1]]  # 下一个合约
    })
    
    return rollover_calendar, main_contract

if individual_contracts:
    rollover_info, main_contract = detect_main_contract_rollover(individual_contracts)

🔍 分析主力合约切换...
📊 合约活跃度 (平均成交量):
   M2301: 375,148
   M2305: 278,055
   M2309: 432,412
   M2401: 480,373
   M2405: 450,806
   M2409: 583,020
   M2501: 454,623
🏆 当前主力合约: M2409


In [9]:
class AKShareFuturesDataProvider:
    """基于 AKShare 的期货数据提供器"""
    
    def __init__(self, symbol="M"):
        self.symbol = symbol  # M=豆粕, A=豆一, Y=豆油, CF=棉花, etc.
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
    
    def get_available_contracts(self):
        """获取可用的期货合约列表"""
        try:
            # 这里需要根据 AKShare 的最新 API 调整
            contracts = ak.tool_trade_date_hist_sina()  # 获取交易日历
            self.logger.info(f"获取到 {len(contracts)} 个可用合约")
            return contracts
        except Exception as e:
            self.logger.error(f"获取合约列表失败: {e}")
            return []
    
    def get_contract_data(self, contract_code, start_date=None, end_date=None):
        """获取单个合约的历史数据"""
        try:
            if start_date is None:
                start_date = "20200101"
            if end_date is None:
                end_date = datetime.now().strftime("%Y%m%d")
            
            # 获取期货日线数据
            data = ak.futures_zh_daily_sina(symbol=contract_code)
            
            if not data.empty:
                # 数据清洗
                data['date'] = pd.to_datetime(data['date'])
                data = data[(data['date'] >= pd.to_datetime(start_date)) & 
                           (data['date'] <= pd.to_datetime(end_date))]
                
                self.logger.info(f"获取合约 {contract_code} 数据: {len(data)} 条")
                return data
            else:
                self.logger.warning(f"合约 {contract_code} 无数据")
                return pd.DataFrame()
                
        except Exception as e:
            self.logger.error(f"获取合约 {contract_code} 数据失败: {e}")
            return pd.DataFrame()
    
    def get_main_contract_data(self, start_date=None, end_date=None):
        """获取主力连续合约数据"""
        try:
            main_symbol = f"{self.symbol}0"  # 主力连续合约
            data = ak.futures_main_sina(symbol=main_symbol)
            
            self.logger.info(f"获取主力连续合约数据: {len(data)} 条")
            return data
        except Exception as e:
            self.logger.error(f"获取主力连续合约失败: {e}")
            return pd.DataFrame()
    
    def build_multi_contract_spread(self, near_months=1, far_months=2):
        """构建多合约价差数据"""
        # 获取当前年月
        current_date = datetime.now()
        current_year = current_date.year
        current_month = current_date.month
        
        # 生成合约代码
        contracts = []
        for i in range(6):  # 获取未来6个月的合约
            target_month = current_month + i
            target_year = current_year
            
            if target_month > 12:
                target_month -= 12
                target_year += 1
            
            # 豆粕通常是1,5,9月份的合约最活跃
            if target_month in [1, 5, 9]:
                contract_code = f"{self.symbol}{target_year-2000:02d}{target_month:02d}"
                contracts.append(contract_code)
        
        self.logger.info(f"尝试获取合约: {contracts}")
        
        # 获取各合约数据
        contract_data = {}
        for contract in contracts[:4]:  # 只取前4个合约
            data = self.get_contract_data(contract)
            if not data.empty:
                contract_data[contract] = data
        
        return self.create_calendar_spread_from_contracts(contract_data)
    
    def create_calendar_spread_from_contracts(self, contract_data):
        """从多个合约数据创建日历价差"""
        if len(contract_data) < 2:
            self.logger.error("需要至少2个合约数据")
            return pd.DataFrame()
        
        # 选择前两个合约
        contract_names = sorted(list(contract_data.keys()))
        near_contract = contract_names[0]
        far_contract = contract_names[1]
        
        self.logger.info(f"创建价差: {near_contract} vs {far_contract}")
        
        # 合并数据
        near_data = contract_data[near_contract].set_index('date')['close']
        far_data = contract_data[far_contract].set_index('date')['close']
        
        # 对齐日期
        spread_data = pd.DataFrame({
            'NEAR': near_data,
            'FAR': far_data
        }).dropna()
        
        spread_data['SPREAD'] = spread_data['FAR'] - spread_data['NEAR']
        
        self.logger.info(f"价差数据创建完成: {len(spread_data)} 条记录")
        return spread_data

# 创建数据提供器实例
print("🔧 创建 AKShare 数据提供器...")
akshare_provider = AKShareFuturesDataProvider("M0")  # M=豆粕

print("✅ AKShare 期货数据提供器创建完成")

🔧 创建 AKShare 数据提供器...


NameError: name 'logging' is not defined